<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_200DMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-10-22 16:29:56


In [2]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
mypf = mypf[mypf['Criteria'].isin(['V40','V40N'])]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [4]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [5]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [6]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      #mid_window = 50
      long_window = 200
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_DMA'] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_DMA'])*100/stock_df['200_DMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):
    #date = (datetime.date.today() + datetime.timedelta(days=0))
    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['200_DMA']) & (df_prec_dev['RSI_14'] < 30) ), 1.0, df_prec_dev['Signal'])

    #df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] > df_prec_dev['200_DMA']) & (df_prec_dev['RSI_14'] > 70) ), -1.0, df_prec_dev['Signal'])

    #df_prec_dev['Position'] = df_prec_dev['Signal'].diff()

    #df_pos = df_prec_dev[(df_prec_dev['Position'] >= 1) | (df_prec_dev['Position'] <= -1)]
    df_prec_dev = df_prec_dev[df_prec_dev['Signal'] != 0]
    #df_pos['Prev_Signal'] = df_pos['Signal'].shift(1)
    #df_pos = df_pos[df_pos['Signal'] != df_pos['Prev_Signal']]
    #df_pos.drop('Prev_Signal', axis=1, inplace=True)

    return df_prec_dev.tail(1)

def get_daily_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        df_buy_sell = df_buy_sell.append(df_tmp)


    df_buy_sell['Signal'] = df_buy_sell['Signal'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()


In [7]:
df_daily_buy_sell = get_daily_buy_sell()
df_daily_buy_sell

,Close,200_DMA,Dev%_200,Stock,Max,Min,Gained%,RSI_14,Signal,Criteria
Date,,,,,,,,,,
2022-11-22 00:00:00+05:30,482.0,506.0,-4.82,MARICO.NS,588.25,468.15,2.88,27.0,Buy,V40
2023-01-02 00:00:00+05:30,1094.0,1221.0,-10.39,HAVELLS.NS,1455.98,1093.48,0.06,27.0,Buy,V40
2023-01-09 00:00:00+05:30,266.0,294.0,-9.44,KANSAINER.NS,436.31,247.50,7.58,29.0,Buy,V40N
2023-01-27 00:00:00+05:30,447.0,512.0,-12.67,BERGEPAINT.NS,710.11,447.11,0.00,29.0,Buy,V40
2023-01-30 00:00:00+05:30,2320.0,2409.0,-3.69,TITAN.NS,3339.50,2292.82,1.19,29.0,Buy,V40
2023-01-30 00:00:00+05:30,1432.0,1538.0,-6.89,COLPAL.NS,2110.75,1424.13,0.55,28.0,Buy,V40
2023-01-30 00:00:00+05:30,810.0,815.0,-0.60,ICICIBANK.NS,992.45,803.88,0.77,28.0,Buy,V40
2023-01-31 00:00:00+05:30,2270.0,2476.0,-8.32,PIDILITIND.NS,2889.66,2258.62,0.50,24.0,Buy,V40
2023-02-02 00:00:00+05:30,2688.0,3069.0,-12.42,ASIANPAINT.NS,3543.70,2687.75,0.00,27.0,Buy,V40


In [8]:
df_daily_buy_sell[df_daily_buy_sell['Signal'] == 'Sell'].tail(20)

,Close,200_DMA,Dev%_200,Stock,Max,Min,Gained%,RSI_14,Signal,Criteria
Date,,,,,,,,,,


In [9]:
df_daily_buy_sell[df_daily_buy_sell['Signal'] == 'Buy'].tail(20)

,Close,200_DMA,Dev%_200,Stock,Max,Min,Gained%,RSI_14,Signal,Criteria
Date,,,,,,,,,,
2023-03-29 00:00:00+05:30,590.0,719.0,-18.00,CAPLIPOINT.NS,1113.05,581.25,1.43,29.0,Buy,V40N
2023-03-29 00:00:00+05:30,3944.0,4660.0,-15.35,BAYERCROP.NS,5923.63,3918.08,0.67,29.0,Buy,V40N
2023-03-31 00:00:00+05:30,994.0,1321.0,-24.76,SFL.NS,1972.20,951.40,4.47,29.0,Buy,V40N
2023-03-31 00:00:00+05:30,321.0,407.0,-21.22,SIS.NS,605.18,320.65,0.00,28.0,Buy,V40N
2023-03-31 00:00:00+05:30,3463.0,4144.0,-16.44,PFIZER.NS,6001.52,3438.24,0.72,24.0,Buy,V40
2023-04-26 00:00:00+05:30,1195.0,1452.0,-17.69,INFY.NS,1848.38,1191.15,0.34,24.0,Buy,V40
2023-05-09 00:00:00+05:30,2038.0,2717.0,-24.99,TEAMLEASE.NS,5280.75,2028.50,0.47,26.0,Buy,V40N
2023-05-26 00:00:00+05:30,1768.0,1990.0,-11.16,VINATIORGA.NS,2333.61,1710.02,3.39,27.0,Buy,V40N
2023-07-10 00:00:00+05:30,3139.0,3330.0,-5.73,EICHERMOT.NS,3808.41,2821.26,11.27,25.0,Buy,V40N


In [10]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [11]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)